# Clustering

An important task in unsupervised learning is to *cluster* data. That is, we wish to find groupings of similar data without any knowledge of 'ground truth' labels. Let's explore some methods for doing this.

## $k$-Means Clustering

Suppose that we have a point cloud of data $X = \{\vec{x}_1,\ldots,\vec{x}_N\}$ with each $\vec{x}_j \in \mathbb{R}^d$. Our goal is to divide $X$ into $k$ clusters, where $k$ is some positive integer we choose ahead of time. 

As usual, let's construct some toy data to experiment with.

In [ ]:
from sklearn.datasets import make_blobs
import matplotlib.pyplot as plt

K = 2 # classes
N = 100 # in each class
dimension = 2
X, y = make_blobs(n_samples=N*K, centers=K, n_features=dimension, random_state=12)

plt.scatter(X[:,0],X[:,1]);

We can see visually that our data roughly lies in two clusters. Moreover, we have 'ground truth' labels.

In [ ]:
plt.scatter(X[:,0],X[:,1],c=y);

For the sake of illustration, suppose we have no labels and that our data lives in a high dimension that we can't plot.

The goal is to *partition* $X$ into $\{S_1,\ldots,S_k\}$ disjoint nonempty subsets $S_j \subset X$. Let $\mu_j$ denote the mean of the points in $S_j$; these are called *cluster centers* in this context. We want our partition to minimize the quantity
$$
\sum_{j=1}^k \sum_{\vec{x} \in S_j} \|\vec{x} - \mu_j\|^2.
$$

The idea is that the winning partition has the data clustered as tightly as possible around the $k$ means (hence the name of the algorithm). 

It is not possible to solve for this partition explicitly, so we will search for it iteratively.

Let's write code to compute the $k$-means clustering partition. 

**Spoiler:** This function is built into `scikit-learn`. The point is to build the algorithm ourselves first to understand how it works.

### The $k$-Means Clustering Algorithm

When writing our algorithm, we'll take the opportunity to demonstrate some more useful `numpy` tricks. These will be pointed out as we go along.

In [ ]:
# Import numpy
import numpy as np

#### Step 1: Initialize with Random Cluster Centers

A useful function for this task is `np.random.choice`.

In [ ]:
# Example
np.random.choice(10,3)

In [ ]:
def cluster_centers(X,K):
    return X[np.random.choice(len(X),size=K)]
    # Pull out entries of X given by the random choice of K indices

# Testing
print(cluster_centers(X,2))
print(cluster_centers(X,2))
print(cluster_centers(X,3))

#### Step 2:  Determine Clusters

For each point in $X$, we figure out which cluster center is nearest to it. 

We will employ a useful trick called *numpy broadcasting*. If we apply arithmetic operations to `numpy` arrays of incompatible sizes, numpy broadcasting will make sense of this by 'broadcasting' the smaller array over the larger one. This only works under certain conditions on the sizes, so we have to put some thought into setting it up.

In [ ]:
# Define arrays to test on
A = np.array([[1,2]])
B = np.array([[0,0],[1,1],[2,2]])

It doesn't make sense mathematical to add these different-sized if we think of them as matrices. On the other hand, `numpy` interprets addition as: 'add the row of `A` to *each* row of `B`.

In [ ]:
# It doesn't make sense to add these arrays mathematically
A + B

We can get even trickier by employing the `np.newaxis` function which takes a 1D array to a 2D array, a 2D array to a 3D array, etc. The way that the function affects the array depends on which 'slot' we use it in.

In [ ]:
# Define a new test array
C = np.array([[1,2],[3,4]])
Cnew = C[:,np.newaxis,:]
print(Cnew.shape)
Cnew

The following gives an error. To see the general rules for broadcasting, check here: https://docs.scipy.org/doc/numpy/user/basics.broadcasting.html

In [ ]:
B+C

On the other hand, our reshaped array `Cnew` follows the rules to be broadcast with `B`.

In [ ]:
print((B+Cnew).shape)
B+Cnew

Now let's define our function. The input is the dataset `X` and a collection of cluster centers `centers` (e.g., the output of `cluster_centers(X,K)`). The output is an array indicating the index of the cluster center to which each element $\vec{x}_j$ of $X$ belongs.

In [ ]:
def closest(X, centers):
    distances = np.linalg.norm(X - centers[:, np.newaxis,:], axis=2)
    return np.argmin(distances, axis=0)

In [ ]:
# Testing
centers = cluster_centers(X,2)
closest(X,centers)

### Exercise 

Take a moment to dissect the code of the `closest` function. Try to undertand exactly what is going on with the broadcasting procedure.

In [183]:
centers

array([[-6.36068278,  5.02371339],
       [-6.71451223,  4.45115953]])

In [184]:
centers[:,np.newaxis,:]

array([[[-6.36068278,  5.02371339]],

       [[-6.71451223,  4.45115953]]])

In [186]:
(X - centers[:,np.newaxis,:]).shape

(2, 200, 2)

In [188]:
np.linalg.norm(X - centers[:, np.newaxis,:], axis=2).shape

(2, 200)

#### Step 3: Update Centers

Now we define an 'update' function. The input is our dataset `X` and a set of cluster centers `centers`. The output is a new collection of cluster centers, obtained by
- partitioning the data according to the input cluster centers,
- computing the mean within each cluster.

In [ ]:
def new_centers(X, centers):
    c = closest(X, centers)
    K = len(np.unique(c)) # Determine K by finding the number of labels in c
    return np.array([X[c==k].mean(axis=0) for k in range(K)])

In [ ]:
# Test
new_centers(X,centers)

#### Step 4: Iterate the procedure

We can now write our algorithm. We simply iterate the procedure above until the cluster center updates stop moving.

In [ ]:
def kMeans(X, K, max_iter = 10000):
    # Initializations
    centers = X[np.random.choice(len(X),size=K)]
    iteration = 0
    Delta = 1
    # While loop with a hard limit on number of iterations
    while Delta > .001 and iteration < max_iter:
        moved = new_centers(X,centers)
        Delta = np.linalg.norm( moved - centers )
        iteration = iteration+1
        centers = moved
    print('Iterations to converge: ', iteration)
    labels = closest(X,centers)
    # Output is a tuple
    return centers, labels

Let's test it on our data!

In [ ]:
centers, labels = kMeans(X,2)

In [ ]:
fig = plt.figure(figsize=(10,5))

p1 = fig.add_subplot(1,2,1)
p1.scatter(X[:,0],X[:,1],c=y)
plt.title('Ground Truth')

p2 = fig.add_subplot(1,2,2)
p2.scatter(X[:,0],X[:,1],c=1-labels) # Use 1-labels so the colors match up
p2.scatter(centers[:,0],centers[:,1], marker = '^', c = 'r')
plt.title('KMeans Algorithm')


Looks great! Of course, there is not reason that $k$-Means should perfectly replicate 'ground truth' labels if the data is not truly clustered. 

Some other issues:
- This is a randomly-initialized iterative algorithm and there is no guarantee that we find an absolute minimum!
- We knew that there should be 2 classes ahead of time. The $k$ in $k$-Means is chosen by the user, so it is definitely possible to make the 'wrong' choice.

In [ ]:
centers, labels = kMeans(X,4)

plt.scatter(X[:,0],X[:,1],c=labels) 
plt.scatter(centers[:,0],centers[:,1], marker = '^', c = 'r');

### Exercise

Try your $k$-Means algorithm on examples `X1` through `X4` below. For each example, try several values of $k$.

In [ ]:
X1, y1 = make_blobs(n_samples=500, center_box=(-3,3), centers=3, random_state=6)


X2, y2 = make_blobs(n_samples=1000, centers=4, random_state=1)

xs = np.linspace(0,2*np.pi,500)
X3 = np.array([np.cos(xs),np.sin(xs)]).T

from sklearn.datasets import make_circles
X4, y4 = make_circles(n_samples=500, noise = 0.02, random_state = 3)

fig = plt.figure(figsize=(10,10))

p1 = fig.add_subplot(2,2,1)
p1.scatter(X1[:,0],X1[:,1])
plt.title('Example 1')

p2 = fig.add_subplot(2,2,2)
p2.scatter(X2[:,0],X2[:,1])
plt.title('Example 2')

p3 = fig.add_subplot(2,2,3)
p3.scatter(X3[:,0],X3[:,1])
plt.title('Example 3');

p4 = fig.add_subplot(2,2,4)
p4.scatter(X4[:,0],X4[:,1])
plt.title('Example 4');

In [ ]:
A = X4

centers, labels = kMeans(A,2)

plt.scatter(A[:,0],A[:,1],c=labels) 
plt.scatter(centers[:,0],centers[:,1], marker = '^', c = 'r');

## $k$-Means with SciKit-Learn

As mentioned above, `scikit-learn` has $k$-Means Clustering capability, as well as several useful functions for analyzing the results.

Let's first try the `scikit-learn` implementation on our toy dataset.

In [ ]:
from sklearn.cluster import KMeans
kmeans = KMeans(n_clusters=2).fit(X)

We can extract labels...

In [ ]:
kmeans.labels_

...and cluster centers.

In [ ]:
kmeans.cluster_centers_

These agree with our algorithm.

In [ ]:
centers, labels = kMeans(X,2)

In [ ]:
print(centers)
print(np.linalg.norm(labels - kmeans.labels_))
# Might need to switch to 1-labels above to see the correct result, since we initialize randomly

### $k$-Means on MNIST

Of course we need to try out the algorithm on MNIST!

In [ ]:
from sklearn.datasets import load_digits
MNIST, MNISTlabels = load_digits(return_X_y=True)
# The included option automatically gives us the vectors and labels

Let&rsquo;s cluster.



In [ ]:
kmeans = KMeans(n_clusters=3).fit(MNIST)

When the ground truth labels (namely `MNISTlabels`) are known, there are a variety of metrics that one can use to evaluate the quality of clustering. Let's use the "Adjusted Rand Index" (see https://scikit-learn.org/stable/modules/generated/sklearn.metrics.adjusted_rand_score.html). For this score, random labeling should give something close to 0, perfect labeling gives 1.

In [ ]:
from sklearn.metrics import adjusted_rand_score
adjusted_rand_score(MNISTlabels, kmeans.labels_)

This does a bad job. Of course, 3 clusters is not a good choice for MNIST.

### Exercise

Run $k$-Means clustering on MNIST for a range of choices for $k$, compute adjusted rand scores for each choice, then plot the results. What looks like the best choice for $k$? Does that agree with intuition?

In [ ]:
scores = []
for j in range(1,25):
    kmeans = KMeans(n_clusters=j).fit(MNIST)
    scores.append(adjusted_rand_score(MNISTlabels, kmeans.labels_))

plt.plot(list(range(1,25)),scores)

Let's take a look which digits are getting clustered together.

In [ ]:
kmeans = KMeans(n_clusters=10).fit(MNIST)

digit = 7 # Note: label 7 doesn't necessarily correspond to the digit 7. Label names are random!
images = MNIST[kmeans.labels_ == digit]

fig, axes = plt.subplots(10,10,figsize=(10,10))

for i in range(10):
    for j in range(10):
        axes[i,j].axis('off')
        axes[i,j].imshow(images[i * 10 + j].reshape(8,8),cmap='gray',interpolation='sinc')

plt.show()

## How Many Clusters?

Since we must choose a value of $k$ to run $k$-Means clustering, there is a big questions when doing *unsupervised learning* (i.e., we don't have 'ground truth' labels): what is the correct choice of $k$?

There is no *real* answer, but we can make an educated guess by looking at the *inertia* of the clustering. This is just the value of the function we were optimizing to begin with, evaulated on our clustering partition:
$$
\sum_{j=1}^k \sum_{\vec{x} \in S_j} \|\vec{x} - \mu_j\|^2.
$$

This is computed in `scikit-learn` as follows.

In [ ]:
kmeans = KMeans(n_clusters=3).fit(X)
kmeans.inertia_

Let&rsquo;s plot this within-cluster sum-of-squares for the clusters computed via `KMeans` for multiple choices of `n_clusters`.



In [ ]:
plt.plot(list(range(1,10)), [KMeans(n_clusters=j).fit(X).inertia_ for j in range(1,10)] )
plt.show()

As we increase the number of clusters, the inertia will *always* decrease, so we are not looking for a local min. Instead, we look for an "elbow", where the slope of the intertia curve abruptly changes. If such an elbow appears, this is generally accepted to be an optimal number of clusters. For the dataset `X`, this tells us that the optimal number of clusters is 2, which should agree with our intuition.

### Exercise

Run this "elbow analysis" on examples `X1` through `X4` from above. Can you determine an optimal number of clusters in each case? What if you run the same sort of analysis on the MNIST dataset?

In [ ]:
plt.plot(list(range(1,10)), [KMeans(n_clusters=j).fit(X1).inertia_ for j in range(1,10)] )
plt.show()